<a href="https://colab.research.google.com/github/paulgureghian/Google_Colab_PyTorch_Notebooks/blob/master/Medetec_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
from torch import nn, optim
from collections import OrderedDict
from torch.optim import lr_scheduler 
from torchvision import  models, datasets, transforms

In [0]:
data_dir = '/content/drive/My Drive/Medetec/data/medetec_dataset'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [0]:
training_transforms= transforms.Compose([transforms.RandomRotation(45),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.RandomCrop(224),
                                         transforms.RandomResizedCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.485,0.456,0.406),
                                                              (0.229,0.224,0.255))])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.485,0.456,0.406),
                                                                 (0.229,0.224,0.255))])

testing_transforms = transforms.Compose([transforms.Resize(256),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.485,0.456,0.406),
                                                              (0.229,0.224,0.255))])

training_dataset = datasets.ImageFolder(train_dir, transform=training_transforms)
validation_dataset = datasets.ImageFolder(valid_dir, transform=validation_transforms)
testing_dataset = datasets.ImageFolder(test_dir, transform=testing_transforms)

training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=32, shuffle=True)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=32)
testing_dataloader = torch.utils.data.DataLoader(testing_dataset, batch_size=32) 

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

model = models.vgg19(pretrained=True)

for param in model.parameters():
    param.requires_grad_(False) 
    
model.classifier = nn.Sequential(OrderedDict([
                                ('fc1', nn.Linear(25088, 4096)),
                                ('relu1', nn.ReLU()),                                                             
                                ('fc2', nn.Linear(4096, 15)), 
                                ('output', nn.Softmax(dim=1))
                                ])) 

epochs = 50
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0002)
scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.3) 

model.to(device) 

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [12]:
for epoch in range(epochs):
    scheduler.step()
    train_loss = 0
    
    model.train()
    for inputs, labels in training_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        
        loss = criterion(logps, labels)
        
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item()
        
    model.eval()
    accuracy = 0
    valid_loss = 0
    
    for inputs, labels in validation_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        output = model.forward(inputs)
        
        loss = criterion(output, labels)
        
        valid_loss += loss.item()
        
        ps = torch.exp(output)
        top_ps, top_class = ps.topk(1, dim=1) 
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
    print('Epoch {}/{}.. '.format(epoch+1, epochs),
          'Training loss: {:.3f}.. '.format(train_loss / len(training_dataloader)),
          'Validation loss: {:.3f}.. '.format(valid_loss / len(validation_dataloader)),
          'Validation accuracy: {:.3f}.. '.format(accuracy / len(validation_dataloader)))
          
    model.train()      

Epoch 1/50..  Training loss: 2.625..  Validation loss: 2.606..  Validation accuracy: 0.211.. 
Epoch 2/50..  Training loss: 2.569..  Validation loss: 2.606..  Validation accuracy: 0.211.. 
Epoch 3/50..  Training loss: 2.569..  Validation loss: 2.606..  Validation accuracy: 0.211.. 
Epoch 4/50..  Training loss: 2.530..  Validation loss: 2.605..  Validation accuracy: 0.211.. 
Epoch 5/50..  Training loss: 2.549..  Validation loss: 2.617..  Validation accuracy: 0.191.. 
Epoch 6/50..  Training loss: 2.559..  Validation loss: 2.558..  Validation accuracy: 0.261.. 
Epoch 7/50..  Training loss: 2.521..  Validation loss: 2.570..  Validation accuracy: 0.250.. 
Epoch 8/50..  Training loss: 2.553..  Validation loss: 2.560..  Validation accuracy: 0.253.. 
Epoch 9/50..  Training loss: 2.494..  Validation loss: 2.549..  Validation accuracy: 0.269.. 
Epoch 10/50..  Training loss: 2.505..  Validation loss: 2.566..  Validation accuracy: 0.245.. 
Epoch 11/50..  Training loss: 2.512..  Validation loss: 2.5